In [ ]:
# COSC 522 UTK
# Group 5 - Final Project
# Corey Lovette, Noah Archer, Stephen Williams and Tyler Herndon

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Read the data from the CSV file
dataset = pd.read_csv('Fraud.csv')

# Display basic information about the dataset
print("Dataset Size: ", dataset.size);
print("Dataset Shape: ", dataset.shape);
print("Dataset Columns: ", dataset.columns);
print("Dataset Data Types: ", dataset.dtypes);
print("Missing Values by Column: ", dataset.isnull().sum());
print("Total Frauds and Non-Frauds: ", dataset['isFraud'].value_counts());

# Display a few of the records
dataset.tail();